In [1]:
import face_recognition as fr
import cv2
import numpy as np
import glob
import re

In [16]:
def image_to_embeddings(file):
    """Return the face embeddings from an image"""
    load_image = fr.load_image_file(file)
    return fr.face_encodings(load_image)[0]

def top_3_matches(known_encodings, names, new_image_embed):
    """With list of known image names/embeddings, return 3 closest matches"""
    closests = fr.face_distance(known_encodings, new_image_embed)
    top_3 = np.argpartition(closests, 3)[:3]
    for i in top_3:
        print(names[i] + "\n")

In [3]:
me = image_to_embeddings('/Users/nawjux/Pictures/My dumb face/profile_.jpg')
jules = image_to_embeddings('/Users/nawjux/Desktop/FullSizeRender-1.jpg')

In [12]:
marvel_cast_files = glob.glob('marvel_characters/*.jpg')
marvel_cast_names = [re.sub('.jpg|marvel_characters/', '', image)
                       for image in marvel_cast_files]
marvel_cast_encodings = [image_to_embeddings(image)
                                  for image in marvel_cast_files]

In [17]:
top_3_matches(marvel_cast_encodings, marvel_cast_names, me)

Iron Man

Spider-Man

Drax



In [41]:
video_capture.release()

In [ ]:
video_capture = cv2.VideoCapture(0)

# Create arrays of known face encodings and their names
# known_face_encodings = [
#     encoding_1,

# ]
# known_face_names = [
#     "John",
# ]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = fr.face_locations(rgb_small_frame)
        face_encodings = fr.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
#             matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
#             name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Use the known face with the smallest distance to the new face
            face_distances = fr.face_distance(marvel_cast_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
#             if matches[best_match_index]:
            name = marvel_cast[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
        

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()